## 1.Export

### Get city_T_df

In [1]:
RERUN_TIME_IN_HOURS = 24
WEATHER_FORCAST_TIMEWINDOW = 3

In [2]:
import pandas as pd
import sqlalchemy
sys.path.append('..\keys')
import key

schema="gans_db"   # name of the database you want to use here
host="localhost"        # to connect to your local server
user="root"
password=key.SQL_PASSWORD # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
city_T_df = pd.read_sql_table('city',con=con)

In [3]:
city_T_df

,city_id,city,country,population,lat,lon
0,1,Istanbul,Turkey,15840900,41.0136,28.95500
1,2,Moscow,Russia,12632409,55.7500,37.61670
2,3,London,United Kingdom,8799800,51.5072,-0.12750
3,4,Saint Petersburg,Russia,5376672,59.9500,30.30000
4,5,Berlin,Germany,3677472,52.5167,13.38330
5,6,Madrid,Spain,3305408,40.3833,-3.71667
6,7,Kyiv,Ukraine,2962180,50.4500,30.52330
7,8,Rome,Italy,2749031,41.9000,12.50000
8,9,Baku,Azerbaijan,2303100,40.4000,49.90000
9,10,Bucharest,Romania,2161347,44.4325,26.10390


### Get Weather from API

In [4]:
import requests
#import json
#import math
import numpy as np
def get_cityWeather(city_df): 
    cities_weath = {"city_id":[]
                    ,"forcast_time":[]
                    ,"outlook":[]
                    ,"temperature":[]
                    ,"feels_like":[]
                    ,"wind_speed":[]
                    ,"rain_prob":[]
                    }
    for i in range(len(city_df)):
        lat=city_df.lat[i]
        lon=city_df.lon[i]
        url_weather= f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={key.Weather_API_key}&units=metric"
        weather_response = requests.get(url_weather)
        weather_response.raise_for_status()

        city_json = weather_response.json()
        #API interface provide weather forcast for the next 5 days(per 3 hours)
        #iterate = math.ceil(RERUN_TIME_IN_HOURS/WEATHER_FORCAST_TIMEWINDOW)
        iterate = 8
        for forcast in city_json["list"][:iterate]:
            cities_weath["city_id"].append(city_df.city_id[i])
            cities_weath["forcast_time"].append(forcast.get('dt_txt', pd.NaT))
            cities_weath["temperature"].append(forcast["main"].get('temp', np.NaN))
            cities_weath["outlook"].append(forcast["weather"][0].get('description', "unknown"))
            cities_weath["feels_like"].append(forcast["main"].get('feels_like', np.NaN))
            cities_weath["wind_speed"].append(forcast["wind"].get('speed', np.NaN))
            cities_weath["rain_prob"].append(forcast.get('pop', np.NaN))
    return cities_weath

In [5]:
cities_weath = get_cityWeather(city_T_df)

In [6]:
weather_df = pd.DataFrame(cities_weath)

## 2.Transform

In [7]:
weather_c_df = weather_df.copy()

In [8]:
weather_c_df.forcast_time = pd.to_datetime(weather_c_df.forcast_time)

In [9]:
weather_c_df

,city_id,forcast_time,outlook,temperature,feels_like,wind_speed,rain_prob
0,1,2023-01-13 15:00:00,broken clouds,10.09,9.40,5.36,0.05
1,1,2023-01-13 18:00:00,overcast clouds,9.75,7.10,5.48,0.05
2,1,2023-01-13 21:00:00,overcast clouds,9.30,6.83,4.72,0.00
3,1,2023-01-14 00:00:00,overcast clouds,8.88,6.32,4.67,0.00
4,1,2023-01-14 03:00:00,overcast clouds,8.74,6.34,4.25,0.00
...,...,...,...,...,...,...,...
283,36,2023-01-14 00:00:00,few clouds,-8.70,-15.70,5.47,0.00
284,36,2023-01-14 03:00:00,few clouds,-8.91,-15.91,5.50,0.00
285,36,2023-01-14 06:00:00,few clouds,-8.59,-15.59,5.66,0.00
286,36,2023-01-14 09:00:00,clear sky,-5.39,-12.39,6.75,0.00


In [10]:
weather_c_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   city_id       288 non-null    int64         
 1   forcast_time  288 non-null    datetime64[ns]
 2   outlook       288 non-null    object        
 3   temperature   288 non-null    float64       
 4   feels_like    288 non-null    float64       
 5   wind_speed    288 non-null    float64       
 6   rain_prob     288 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 15.9+ KB


## 3.load

### Load final df to Local MySQL

In [11]:
weather_c_df.to_sql('weather',con=con,if_exists='append',index=False)

288